No idea how to get cloud-less historical landsat data yet.

In [2]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# read a kml file and get the coordinates

In [23]:
shp_path = r'.\files4shTest\shapeFile\Rio_de_Janeiro\Rio_de_Janeiro.shp'

shapefile = gpd.read_file(shp_path)
print(shapefile.shape)

geom = shapefile.iloc[0, :]['geometry']
print('geom',geom)

p1=geom.centroid
print(p1.x,p1.y)

p1.to_file("studyArea.geojson", driver='GeoJSON')

print(shapefile.crs)

(177, 12)
geom POLYGON Z ((-43.17864255863062 -22.90157687956335 0, -43.17670204128276 -22.90106832915569 0, -43.17528185700142 -22.90363105179198 0, -43.17412043805864 -22.9054605176207 0, -43.17348214990605 -22.90738517122853 0, -43.1719563214834 -22.91093258117178 0, -43.17463328535334 -22.91241252727201 0, -43.17584627984972 -22.90886198305033 0, -43.17710710628753 -22.90552088593094 0, -43.17864255863062 -22.90157687956335 0))
-43.1752558600014 -22.906782934461642


AttributeError: 'Point' object has no attribute 'to_file'

# playaround gpd: read a kml file and convert to gee polygon

In [24]:
import geopandas as gpd
import fiona 

# Enable fiona driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

polygon = gpd.read_file(r'.\files\zhengzhou.kml', driver='KML')
# print(polygon)
geom = polygon.iloc[0:1, :]
# print(geom)

'''convert to gee feature'''
jsonDict = eval(geom.to_json())
geojsonDict = jsonDict['features'][0]
print(geojsonDict)

cor=geojsonDict['geometry']['coordinates']
print()
         
aoi = ee.Geometry.Polygon([cor[0][0][0:2], cor[0][1][0:2], cor[0][2][0:2], cor[0][3][0:2],cor[0][4][0:2]], None, False)
print(aoi)
            

{'id': '0', 'type': 'Feature', 'properties': {'Description': '', 'Name': 'zhengzhou'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[113.4428143023446, 34.59865400274417, 0.0], [113.8930630828336, 34.60378289276893, 0.0], [113.8953835475239, 34.86488969739759, 0.0], [113.4355495368252, 34.8620760235706, 0.0], [113.4428143023446, 34.59865400274417, 0.0]]]}}

ee.Geometry({
  "type": "Invocation",
  "arguments": {
    "coordinates": [
      [
        113.4428143023446,
        34.59865400274417
      ],
      [
        113.8930630828336,
        34.60378289276893
      ],
      [
        113.8953835475239,
        34.86488969739759
      ],
      [
        113.4355495368252,
        34.8620760235706
      ],
      [
        113.4428143023446,
        34.59865400274417
      ]
    ],
    "geodesic": false
  },
  "functionName": "GeometryConstructors.Polygon"
})


# playaround gee

In [8]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [9]:
# Display the polygons by adding them to the map.
Map.centerObject(aoi,10)
Map.addLayer(aoi, {'color': 'FF0000'})

In [5]:
Map.add_landsat_ts_gif(label='', start_year=1985, bands=['Red', 'Green','Blue'], frames_per_second=5)# a default example

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\User\Downloads\landsat_ts_sqt.gif
Adding animated text to GIF ...
Adding GIF to the map ...


In [10]:
#To create a Landsat timelapse animation and add it to the Map:  
Map.add_landsat_ts_gif(label='roi', roi=aoi, start_year=1985, bands=['Red', 'Green','Blue'], frames_per_second=1)#for the selected area
#

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\User\Downloads\landsat_ts_roi.gif
Adding animated text to GIF ...
Adding GIF to the map ...


# The images in the gif is very bad, some improvements need to be done for the data to be useful

In [76]:
imgC=geemap.landsat_timeseries(roi=aoi, start_year=1984, end_year=2019, start_date='06-10', end_date='09-20')
#print(imgC.getInfo())
count = imgC.size().getInfo()
print("Count: ", count)
vis = {'bands': ['Red', 'Green','Blue']}
Map.addLayer(imgC,vis)

Count:  36


In [ ]:
geemap.ee_export_image_collection(ee_object, output, scale, crs, region, file_per_band)

# 

In [ ]:
import os

# aoi = ee.Geometry.Polygon(
#   [[[-110.8, 44.7],
#     [-110.8, 44.6],
#     [-110.6, 44.6],
#     [-110.6, 44.7]]], None, False)

loc = ee.Geometry.Point(-99.2222, 46.7816)
collection = ee.ImageCollection('USDA/NAIP/DOQQ') \
    .filterBounds(loc) \
    .filterDate('2008-01-01', '2020-01-01') \
    .filter(ee.Filter.listContains("system:band_names", "N"))

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

print(collection.aggregate_array('system:index').getInfo())

geemap.ee_export_image_collection(collection, out_dir=out_dir)

#https://github.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb